# Machine learning customer segmentation project
In this project, I will build a machine learning model for customer segmentation. It involves unsupervised learning, using an unlabeled training set for clustering. I intend to perform clustering to summarize customer segments of a retail store, identifying an ideal client group and building a loyalty program.

I will use the following pipeline, based on CRISP-DM framework:

1. Define the business problem.
2. Initial data understanding.
3. Exploratory data analysis and feature engineering.
4. Data cleaning and preprocessing.
5. Group customers into clusters, modelling.
6. Interpret the results, analysing the groups created (profiling). 
7. Provide marketing suggestions and the loyalty program.
8. Estimate financial results.

In this notebook, I will be covering steps 4 to 8 of the pipeline above. The main objective here is to segment our customers, studying the embedding space, applying techniques such as dimensionality reduction and comparing and evaluating different clustering algorithms in order to make the best segmentation. Moreover, profiles will be created and a loyalty program will be designed, such that concrete financial results will be calculated at the end.

### Importing the libraries

In [1]:
# Data manipulation and visualization.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling.
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Utils.
from scripts.modelling_utils import *

### Reading the data

In [2]:
df = pd.read_csv('data/marketing_campaign.csv', sep='\t')

In [3]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns.')

The dataset has 2240 rows and 29 columns.


### Feature engineering
Obtaining the final dataset for clustering, applying all the feature engineering and data cleaning at once.

In [5]:
clustering_df = feature_engineering(df)

In [6]:
clustering_df.columns

Index(['education', 'marital_status', 'income', 'recency', 'mntwines',
       'mntfruits', 'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts',
       'mntgoldprods', 'numdealspurchases', 'numwebpurchases',
       'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth',
       'total_accepted_cmp', 'children', 'age', 'relationship_duration',
       'frequency', 'monetary', 'avg_purchase_value'],
      dtype='object')

In [7]:
print(f'The clustering dataset has {clustering_df.shape[0]} rows and {clustering_df.shape[1]} columns.')

The clustering dataset has 2228 rows and 22 columns.


# 4. Data cleaning and preprocessing
- In order to fit machine learning algorithms, there are some transformations we need to apply to the data
- Outliers: I have already treated the outliers, removing those representing inconsistent information.
- Missing values: I will replace the missing values in income column using KNNImputer, in order to avoid biased clustering results. It will take the mean income from the nearest n_neighbors for imputing.
- Numerical features: Some clustering algorithms are sensitive to feature scaling because they use distance calculations such as euclidean distance for training and predicting. Thus, I will apply StandardScaler to numeric attributes.
- Categorical features: I intend to apply OrdinalEncoder to the categorical features because education has an ordinal relationship and marital status has only two categories, which will be encoded as 0 and 1. Moreover, I will scale these features after encoding because some clustering algorithms are sensitive to it, due to distance calculations.

Checking missing values.

In [8]:
clustering_df.isna().sum()

education                 0
marital_status            0
income                   23
recency                   0
mntwines                  0
mntfruits                 0
mntmeatproducts           0
mntfishproducts           0
mntsweetproducts          0
mntgoldprods              0
numdealspurchases         0
numwebpurchases           0
numcatalogpurchases       0
numstorepurchases         0
numwebvisitsmonth         0
total_accepted_cmp        0
children                  0
age                       0
relationship_duration     0
frequency                 0
monetary                  0
avg_purchase_value        2
dtype: int64

I will build the preprocessor below, following the techniques mentioned above.

In [9]:
numerical_features = clustering_df.select_dtypes('number').columns.to_list()
categorical_features = clustering_df.select_dtypes('object').columns.to_list()

print(f'There are {len(numerical_features)} numerical features. They are: ')
print(numerical_features)
print(f'There are {len(categorical_features)} categorical features. They are: ')
print(categorical_features)

There are 20 numerical features. They are: 
['income', 'recency', 'mntwines', 'mntfruits', 'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts', 'mntgoldprods', 'numdealspurchases', 'numwebpurchases', 'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth', 'total_accepted_cmp', 'children', 'age', 'relationship_duration', 'frequency', 'monetary', 'avg_purchase_value']
There are 2 categorical features. They are: 
['education', 'marital_status']


In [10]:
num_pipeline = Pipeline(steps=[
    ('knn_imputer', KNNImputer()),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoder()),
    ('std_scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_features),
        ('cat', cat_pipeline, categorical_features)
    ]
)

Fitting the preprocessor and converting to DataFrame in order to perform embedding space study.

In [11]:
prepared_df = pd.DataFrame(preprocessor.fit_transform(clustering_df))

In [12]:
prepared_df.columns = clustering_df.columns

In [13]:
prepared_df.head()

,education,marital_status,income,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,...,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,total_accepted_cmp,children,age,relationship_duration,frequency,monetary,avg_purchase_value
0,0.317619,0.307307,0.979727,1.550286,1.755827,2.456149,1.471551,0.842345,0.358429,1.405182,...,0.689529,-0.439298,-1.267017,1.015275,1.501310,1.039875,1.684878,1.129377,-0.892864,1.345208
1,-0.253280,-0.383626,-0.873488,-0.637803,-0.730377,-0.651835,-0.633150,-0.731880,-0.169383,-1.118159,...,-0.139543,-0.439298,1.393599,1.271558,-1.420017,-1.091331,-0.962467,-0.983387,-0.892864,1.345208
2,0.969888,-0.798186,0.359019,0.569418,-0.173283,1.340931,-0.149310,-0.040757,-0.697195,1.405182,...,-0.554079,-0.439298,-1.267017,0.331856,0.040647,0.810670,0.284616,0.155932,-0.892864,-0.743380
3,-1.206779,-0.798186,-0.873488,-0.562352,-0.665920,-0.505577,-0.584766,-0.751077,-0.169383,-0.757682,...,0.274993,-0.439298,0.063291,-1.291266,-1.420017,-0.796447,-0.919177,-0.908784,-0.892864,-0.743380
4,0.325122,1.550987,-0.392365,0.418516,-0.214720,0.152585,-0.004159,-0.559099,1.414053,0.323750,...,-0.139543,-0.439298,0.063291,-1.034984,-1.420017,0.825414,-0.304793,-0.361616,0.572012,-0.743380


### Dimensionality reduction (PCA)
I apply PCA, a dimensionality reduction technique for some reasons:

- By embedding the original customer data into a lower dimensional space, clustering algorithms can be applied more effectively.
- Dimensionality reduction will help us removing noise in the data. Clustering algorithms can be pretty sensitive to outliers and noise.
- By embedding the original customer data into a lower dimensional space, it will be possible to effectively visualize the clusters created. This is pretty useful, since it is tough to evaluate clustering algorithms, even using metrics such as silhouette score.